# Block descent for optimizing group knockoffs

There are 2 ways for optimizing the group knockoff problem. This notebook tests the block update strategy where we update an entire block at once. Two concerns arise 

+ Which solver (MOSEK, Hypatia.jl, SCS, SDPT3, Convex.jl) should we use to solve each individual block?
+ How to maintain inequality constraints after updating a block? Can we use Woodbury?

In [18]:
using Revise
using Knockoffs
using BlockDiagonals
using LinearAlgebra
using Hypatia
using JuMP
using ToeplitzMatrices
using StatsBase
using CovarianceEstimation
# ENV["COLUMNS"] = 240

#
# test problem
#
p = 200
m = 5
group_sizes = [5 for i in 1:div(p, 5)] # each group has 5 variables
groups = vcat([i*ones(g) for (i, g) in enumerate(group_sizes)]...) |> Vector{Int}
# Σ = simulate_block_covariance(groups, 0.75, 0.25)
# Σ = Matrix(SymmetricToeplitz(0.4.^(0:(p-1)))) # true covariance matrix
# Σ = Matrix(SymmetricToeplitz((-0.4).^(0:(p-1)))) # true covariance matrix
# Σ = simulate_block_covariance(groups, 0.5, 0.6) # true covariance matrix
# Σ = simulate_AR1(p, a=3, b=1) # true covariance matrix
x = randn(p, p)
Σ = x'*x
StatsBase.cov2cor!(Σ, std(Σ, dims=1))
# Σ

200×200 Matrix{Float64}:
  1.0        -0.0428806    -0.0194029   …   0.0245423     0.0531279
 -0.0428806   1.0           0.0552612      -0.0348509     0.0214462
 -0.0194029   0.0552612     1.0            -0.0504266     0.00472199
  0.0122933  -0.00823334   -0.0341107      -0.0307394    -0.0254613
 -0.0268985  -0.0285586     0.0143846      -0.00308192    0.0121367
  0.0368042   0.0133676    -0.0427958   …   0.017744      0.034376
  0.0115276   0.0379797     0.00494121      0.0527435     0.0444482
  0.0382757  -0.0321222    -0.0070667       0.00754915   -0.000707287
 -0.109608    0.0311677    -0.0147732       0.000138236   0.0257921
 -0.0209923  -0.0321311    -0.0122957       0.0130811    -0.00240639
  0.0166066   0.0148456     0.0115445   …   0.032001     -0.0162767
 -0.0164577  -0.0147782    -0.0233369      -0.04476      -0.0209731
  0.0742065  -0.024809      0.0186569      -0.00133517    0.0276087
  ⋮                                     ⋱                
 -0.0172463   0.0335939     0.

## Suboptimal SDP group knockoffs

In [19]:
@time Gsdp_subopt, _ = solve_s_group(
    Σ, groups, :sdp_subopt,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

  1.170926 seconds (245.04 k allocations: 57.439 MiB)


In [20]:
@time Gsdp_subopt_correct, _ = solve_s_group(
    Σ, groups, :sdp_subopt_correct,
    m = m,          # number of knockoffs per variable to generate
    verbose=true);  # whether to print informative intermediate results

 31.662152 seconds (536.10 k allocations: 1.713 GiB, 1.26% gc time)


In [21]:
sum(Gsdp_subopt), sum(Gsdp_subopt_correct)

(115.59756069304356, 115.59687211629944)

## Use @objective with slack variables or NLobjective directly?

In [11]:
function slack_SDP_single_block(
    Σ11::AbstractMatrix,
    ub::AbstractMatrix;  # this is upper bound, equals [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false, iter_limit=100) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    # slack variables to handle absolute value in obj 
    @variable(model, U[1:p, 1:p], Symmetric)
    for i in 1:p, j in i:p
        @constraint(model, Σ11[i, j] - S[i, j] ≤ U[i, j])
        @constraint(model, -U[i, j] ≤ Σ11[i, j] - S[i, j])
    end
    @objective(model, Min, sum(U))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S)
end

using SCS
function NLObj_SDP_single_block(
    Σ11::AbstractMatrix,
    ub::AbstractMatrix;  # this is upper bound, equals [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=SCS.Optimizer() # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    @NLobjective(model, Min, sum(abs(Σ11[i, j] - S[i, j]) for i in 1:p, j in 1:p))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S)
end

NLObj_SDP_single_block (generic function with 1 method)

In [12]:
m = 1

# initialize with equicorrelated solution
# Sequi, γ = solve_s_group(Σ, groups, :equi)
Sequi = zeros(p, p)

# form constraints for block 1
Σ11 = Σ[1:5, 1:5]
A = (m+1)/m * Σ
D = A - Sequi
A11 = @view(A[1:5, 1:5])
D12 = @view(D[1:5, 6:end])
D22 = @view(D[6:end, 6:end])
ub = A11 - D12 * inv(D22) * D12'

# solve 
@time slack = slack_SDP_single_block(Σ11, ub)
@time nl = NLObj_SDP_single_block(Σ11, ub);

  0.209440 seconds (458.88 k allocations: 24.826 MiB, 57.65% compilation time)


LoadError: The solver does not support nonlinear problems (i.e., NLobjective and NLconstraint).

In [22]:
using JuMP
using MosekTools

model = Model(Mosek.Optimizer)

@variable(model, 2.5 ≤ z1 ≤ 5.0)
@variable(model, -1.0 <= z2 <= 1.0)
@objective(model, Min, abs(z1+5.0) + abs(z2-3.0))

LoadError: Constraints of type MathOptInterface.VariableIndex-in-MathOptInterface.GreaterThan{Float64} are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

Try block descent

In [6]:
@time Ssdp, _ = solve_s_group(
    Σ, groups, :sdp,
    m = 1, 
    verbose=true,
    niter=5
);

Init obj = 651.4494943754169
Iter 1 δ = 0.9536228199702621, obj = 430.76840338555894


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 2 δ = 0.826901097351207, obj = 426.18848412004144


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 3 δ = 0.029548257620773677, obj = 425.975056006773
Iter 4 δ = 0.5597317692795303, obj = 419.2123948332009


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106
┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 5 δ = 0.035389868543257264, obj = 419.0889051776713
 44.680826 seconds (127.32 M allocations: 7.957 GiB, 3.62% gc time, 77.06% compilation time)


In [3]:
@time Sequi, _ = solve_s_group(
    Σ, groups, :equi,
    m = 1, 
);

  0.139828 seconds (439.08 k allocations: 24.683 MiB, 86.85% compilation time)


In [8]:
@time Ssdp_subopt, _ = solve_s_group(
    Σ, groups, :sdp_subopt,
    m = 1, 
);

  1.880804 seconds (856.86 k allocations: 91.592 MiB, 7.57% gc time, 35.69% compilation time)


In [4]:
@time Sme, _ = solve_s_group(
    Σ, groups, :maxent,
    m = 1, 
    verbose=true
);

initial obj = -1001.2975818995944
Iter 1: obj = -676.9797433696642, δ = 0.7148761215068528, t1 = 0.14, t2 = 0.04, t3 = 0.0
Iter 2: obj = -667.02018374056, δ = 0.1389593291910438, t1 = 0.15, t2 = 0.05, t3 = 0.0
Iter 3: obj = -665.1059850352791, δ = 0.021527165678117164, t1 = 0.16, t2 = 0.07, t3 = 0.0
Iter 4: obj = -664.2229660971566, δ = 0.00986453850330372, t1 = 0.17, t2 = 0.08, t3 = 0.0
  0.553342 seconds (1.08 M allocations: 58.418 MiB, 5.56% gc time, 79.26% compilation time)


In [9]:
sum(Sequi), sum(Ssdp), sum(Sme), sum(Ssdp_subopt)

(31.681698156089773, 312.2139302866235, 66.42467742735388, 223.5460638065474)

In [11]:
Ssdp

200×200 Matrix{Float64}:
 1.0       0.559985  0.551354  0.472459  …  0.0        0.0       0.0
 0.559985  1.0       0.983603  0.842856     0.0        0.0       0.0
 0.551354  0.983603  1.0       0.856051     0.0        0.0       0.0
 0.472459  0.842856  0.856051  1.0          0.0        0.0       0.0
 0.37607   0.670901  0.681403  0.795189     0.0        0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 0.0       0.0       0.0       0.0          0.0        0.0       0.0
 ⋮                                       ⋱                       
 0.0       0

In [10]:
Ssdp_subopt

200×200 Matrix{Float64}:
 1.0       0.559985  0.551354  0.472459  …  0.0       0.0       0.0
 0.559985  1.0       0.983603  0.842856     0.0       0.0       0.0
 0.551354  0.983603  1.0       0.856051     0.0       0.0       0.0
 0.472459  0.842856  0.856051  1.0          0.0       0.0       0.0
 0.37607   0.670901  0.681403  0.795189     0.0       0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 0.0       0.0       0.0       0.0          0.0       0.0       0.0
 ⋮                                       ⋱                      
 0.0       0.0       0.0  

In [222]:
[vec(Sequi) vec(Ssdp) vec(Sme)]

40000×3 Matrix{Float64}:
 0.0626931  1.0       0.555897
 0.0417102  0.665309  0.184368
 0.0302182  0.482002  0.0209709
 0.0288381  0.459989  0.00883674
 0.0247707  0.39511   0.00441798
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 ⋮                    
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0130953  0.39511   0.000925487
 0.0167721  0.593875  0.00262455
 0.0186497  0.819727  0.00893611
 0.0515229  0.858956  0.169722
 0.0626931  1.0       0.32336

What if we run group knockoffs when all groups are singletons?

In [127]:
groups = collect(1:p)

# group SDP with single groups
@time Ssdp, _ = solve_s_group(
    Σ, groups, :sdp_block,
    m = 1, 
    verbose=true,
    niter=5
)

# non-grouped SDP
@time ssdp = solve_s(
    Symmetric(Σ), :sdp,
    m = 1, 
    verbose=true,
);

g = 1
A11 = [2.0;;]
size(A11) = (1, 1)


LoadError: UndefVarError: fff not defined

Here Ssdp should be diagonal matrix.

In [87]:
Ssdp

200×200 Matrix{Float64}:
 1.0  0.0  0.0  0.0       0.0      0.0       …  0.0       0.0       0.0
 0.0  1.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  1.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.995473  0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.63106  0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.554254  …  0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0       …  0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 ⋮                                 ⋮   

In [86]:
[ssdp diag(Ssdp)]

200×2 Matrix{Float64}:
 1.0         1.0
 0.468434    1.0
 0.86833     1.0
 0.914276    0.995473
 3.36244e-8  0.63106
 0.5571      0.554254
 4.10019e-8  0.298628
 0.236487    0.265939
 0.0911048   0.198897
 0.0657967   0.186742
 0.198103    0.351992
 0.29984     0.611173
 0.694426    0.84924
 ⋮           
 0.0318133   0.047177
 0.011571    0.0522678
 0.599472    0.822529
 0.412487    0.965399
 0.819467    0.90014
 1.32504e-8  0.398157
 0.354499    0.437457
 0.403364    0.483912
 1.07897e-8  0.136476
 0.115996    0.112647
 1.02813e-8  0.181416
 0.376427    0.371388

In [89]:
eigmin(2Σ - Diagonal(ssdp))

-4.075475130804836e-9

In [91]:
eigmin(2Σ - Ssdp)

-0.3749872292309029

## How to permute things

In [224]:
S1 = reshape(1:16, 4, 4) |> Matrix
S2 = reshape(1:9, 3, 3) |> Matrix
S3 = reshape(1:4, 2, 2) |> Matrix
S = BlockDiagonal([S1, S2, S3])

9×9 BlockDiagonal{Int64, Matrix{Int64}}:
 1  5   9  13  0  0  0  0  0
 2  6  10  14  0  0  0  0  0
 3  7  11  15  0  0  0  0  0
 4  8  12  16  0  0  0  0  0
 0  0   0   0  1  4  7  0  0
 0  0   0   0  2  5  8  0  0
 0  0   0   0  3  6  9  0  0
 0  0   0   0  0  0  0  1  3
 0  0   0   0  0  0  0  2  4

In [233]:
perm = collect(1:9)
g = 2
offset = 7
cur_idx = offset + 1:offset + g
for i in 1:offset
    perm[g+i] = i
end
perm[1:g] .= cur_idx
perm

9-element Vector{Int64}:
 8
 9
 1
 2
 3
 4
 5
 6
 7

In [235]:
Spermuted = S[perm, perm]

9×9 Matrix{Int64}:
 1  3  0  0   0   0  0  0  0
 2  4  0  0   0   0  0  0  0
 0  0  1  5   9  13  0  0  0
 0  0  2  6  10  14  0  0  0
 0  0  3  7  11  15  0  0  0
 0  0  4  8  12  16  0  0  0
 0  0  0  0   0   0  1  4  7
 0  0  0  0   0   0  2  5  8
 0  0  0  0   0   0  3  6  9

In [236]:
iperm = invperm(perm)
Spermuted[iperm, iperm]

9×9 Matrix{Int64}:
 1  5   9  13  0  0  0  0  0
 2  6  10  14  0  0  0  0  0
 3  7  11  15  0  0  0  0  0
 4  8  12  16  0  0  0  0  0
 0  0   0   0  1  4  7  0  0
 0  0   0   0  2  5  8  0  0
 0  0   0   0  3  6  9  0  0
 0  0   0   0  0  0  0  1  3
 0  0   0   0  0  0  0  2  4

## Solving Convex problems

Try solving first block

In [3]:
#
# test problem
#
p = 15
group_sizes = [5 for i in 1:div(p, 5)] # each group has 5 variables
groups = vcat([i*ones(g) for (i, g) in enumerate(group_sizes)]...) |> Vector{Int}
# Σ = simulate_block_covariance(groups, 0.75, 0.25)
# Σ = Matrix(SymmetricToeplitz(0.4.^(0:(p-1)))) # true covariance matrix
# Σ = Matrix(SymmetricToeplitz((-0.4).^(0:(p-1)))) # true covariance matrix
Σ = simulate_AR1(p, a=3, b=1) # true covariance matrix

"""
    solve_full_SDP
"""
function solve_full_SDP(
    Σ11::AbstractMatrix, # correlation matrix
    ub::AbstractMatrix; # this is [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    # slack variables to handle absolute value in obj 
    @variable(model, U[1:p, 1:p], Symmetric)
    for i in 1:p, j in i:p
        @constraint(model, Σ11[i, j] - S[i, j] ≤ U[i, j])
        @constraint(model, -U[i, j] ≤ Σ11[i, j] - S[i, j])
    end
#     @constraint(model, U in PSDCone())      #### is this constraint needed????
    @objective(model, Min, sum(U))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S), objective_value(model)
end

function solve_full_SDP_old(
    Σ11::AbstractMatrix, # correlation matrix
    ub::AbstractMatrix; # this is [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, 0 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    @objective(model, Min, sum(Σ11 - S))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S)
end

solve_full_SDP_old (generic function with 1 method)

In [4]:
m = 1

# initialize with equicorrelated solution
# Sequi, γ = solve_s_group(Σ, groups, :equi)
Sequi = zeros(p, p)

# form constraints for block 1
Σ11 = Σ[1:5, 1:5]
A = (m+1)/m * Σ
D = A - Sequi
A11 = @view(A[1:5, 1:5])
D12 = @view(D[1:5, 6:end])
D22 = @view(D[6:end, 6:end])
ub = A11 - D12 * inv(D22) * D12'

# solve 
@time S1_new, obj = solve_full_SDP(Σ11, ub)
@time S1_old = solve_full_SDP_old(Σ11, ub);

 38.529136 seconds (139.35 M allocations: 8.343 GiB, 4.92% gc time, 99.33% compilation time)
  0.162872 seconds (511.00 k allocations: 30.119 MiB, 13.71% gc time, 91.17% compilation time)


In [163]:
obj, sum(abs.(Σ11 - S1_new))

(2.696897785289176e-9, 9.016139224105046e-10)

In [160]:
eigmin(S1_new)

0.0790755108995826

In [161]:
eigmin(ub)

0.1581510007508361

Lets eyeball the result first

In [94]:
S1_new

5×5 Matrix{Float64}:
 1.0     0.4    0.16  0.064  0.0256
 0.4     1.0    0.4   0.16   0.064
 0.16    0.4    1.0   0.4    0.16
 0.064   0.16   0.4   1.0    0.4
 0.0256  0.064  0.16  0.4    1.0

In [95]:
S1_old

5×5 Matrix{Float64}:
 0.789886  0.747998  0.769968  0.644723  0.319591
 0.747998  0.962284  0.938245  0.82901   0.502877
 0.769968  0.938245  0.982537  0.854499  0.525783
 0.644723  0.82901   0.854499  0.879502  0.417846
 0.319591  0.502877  0.525783  0.417846  0.477052

In [96]:
sum(abs.(S1_new)), sum(abs.(S1_old))

(9.467199998341476, 17.192337396303046)

Check objective

In [162]:
sum(abs.(Σ11 - S1_new)), sum(abs.(Σ11 - S1_old))

(9.016139224105046e-10, 8.45111995244404)

Check constraints

In [98]:
isposdef(S1_old), isposdef(S1_new)

(true, true)

In [99]:
eigvals(ub - S1_old)

5-element Vector{Float64}:
 7.309755484523514e-9
 0.7520353697688693
 0.9080664264148712
 1.4359163255019285
 2.0506742273768

In [100]:
eigvals(ub - S1_new)

5-element Vector{Float64}:
 0.32129497088496595
 0.4766120211625212
 0.643214501447116
 1.0285108585981437
 1.7683208812336677

### Singleton groups?

In [105]:
groups[2:5] .= 2
groups

15-element Vector{Int64}:
 1
 2
 2
 2
 2
 2
 2
 2
 2
 2
 3
 3
 3
 3
 3

In [132]:
m = 1

# initialize with equicorrelated solution
Sequi, γ = solve_s_group(Σ, groups, :equi)

# form constraints for block 1
g = 1
Σ11 = @view(Σ[1, 1])
A = (m+1)/m * Σ
D = A - Sequi
Σ11 = @view(Σ[1:g, 1:g])
A11 = @view(A[1:g, 1:g])
D12 = @view(D[1:g, g + 1:end])
D21 = @view(D[g + 1:end, 1:g])
D22 = @view(D[g + 1:end, g + 1:end])
ub = Symmetric(A11 - D21' * inv(D22) * D21)

# solve 
@time S1_new, U = solve_full_SDP(Σ11, ub)
# @time S1_old = solve_full_SDP_old(Σ11, ub);

  0.066523 seconds (118.17 k allocations: 6.466 MiB, 89.63% compilation time)


In [134]:
S1_new, ub

([0.8644309427462861;;], [0.8644309426414065;;])

## Woodbury updates

Suppose we have 

\begin{align*}
    S = 
    \begin{bmatrix}
        S^{(1)} & & \\
        & S^{(2)} & \\
        & & S^{(3)}
    \end{bmatrix}
\end{align*}
And we want to udpate $S^{(1)}$

$$(A + UC)^{-1} = A^{-1} - A^{-1}U(I + VA^{-1}U)^{-1}VA^{-1}$$

In [1]:
using WoodburyMatrices

In [ ]:
A = 